# Perform single cell analysis, integrating with CRISPRa screen

In [ ]:
!lamin login testuser2

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import scanpy as sc

In [ ]:
ln.track()

In [ ]:
file_hits = ln.File.filter(description="hits from schmidt22 crispra GWS").one()

In [ ]:
file_ps = ln.File.filter(description__icontains="perturbseq", suffix=".h5ad").one()

## Link experimental metadata to the datasets

In [ ]:
efo = lb.ExperimentalFactor.lookup()

In [ ]:
file_hits.add_labels(efo.grna_seq, "assay")

In [ ]:
file_hits.add_labels(efo.interferon_gamma, "readout")

In [ ]:
file_ps.add_labels(efo.perturb_seq, "assay")

In [ ]:
file_ps.add_labels(efo.single_cell_rna_sequencing, "readout")

In [ ]:
is_experiment = ln.Label.filter(name="is_experiment").one()
is_biosample = ln.Label.filter(name="is_biosample").one()

In [ ]:
exp1 = is_experiment.children.filter(description__contains="Genome-wide")
exp2 = is_experiment.children.filter(description__contains="Perturb-seq")

In [ ]:
biosample = is_biosample.children.get(name="S001")

In [ ]:
file_hits.add_labels(exp1, "experiment")
file_ps.add_labels(exp2, "experiment")
file_hits.add_labels(biosample, "biosample")
file_ps.add_labels(biosample, "biosample")

## Load in the scRNA-seq count matrix generated from the BFX pipeline

In [ ]:
adata = file_ps.load()

In [ ]:
screen_hits = file_hits.load()

## Perform single cell analysis, integrating the CRISPR screen data

In [ ]:
sc.tl.score_genes(adata, adata.var_names.intersection(screen_hits.index).tolist())

In [ ]:
sc.pl.umap(adata, color="cluster_name")

In [ ]:
sc.pl.umap(adata, color="score")

In [ ]:
sc.pl.matrixplot(adata, groupby="cluster_name", var_names=["score"])

## Ingest the processed dataset into LaminDB

In [ ]:
file = ln.File(
    adata, key="perturbseq_analyzed.h5ad", description="analyzed data of perturbseq"
)

In [ ]:
file.save()

In [ ]:
file.view_lineage()